# Python Workshop
# Session 2: Working with Structured Data

Stefan Scholz

In this second session we will build an exemplary **data pipeline** with Python. For this, we will work with **external packages** to **import** and **clean data**, then perform **simple analyses** and finally make **visualizations**.


## 2.1 Data Sources

In practice, there are many different data sources available where you can get data from. Each source has its own advantages and disadvantages. So far we have worked a lot with variables in our code, but they are not suitable for storing big amounts of data. That is why we also used files which can store more data, but they are difficult to exchange. This is where application programming interface (API) have emerged that provide an interface for exchanging data between clients and servers. At the same time, we will look at how to get data from websites. Behind APIs and websites, there is usually a database system in which all data is systematically stored and made available. These databases can also be used directly from Python.

Below is a short (and probably incomplete) list of data sources.

| Source | Description |
| -------- | ------- |
| Variable | reserved memory location to store data |
| File | physical storage to store data |
| Website | online available web resources |
| API | interface for exchanging data between clients and servers |
| Database | organized storage and access with software |


We will begin to work with files from the [tree cadastre of the city of Konstanz](https://offenedaten-konstanz.de/dataset/baumkataster-konstanz).

## 2.2 Data Handling



### Manual

To read and write data to files, the module `csv` provides a lot of functionalities. This way you can get data into and out of your program. In CSV files you can find tabular data as comma-separated values. Each line is a data record. And each record has one or more fields which are comma separated.

Let us first import the module.

In [ ]:
import csv

Suppose we want to work with the files from the tree cadastre of the city of Konstanz. You can load the files by opening the file in read mode `r` and a reader is created like `csv.reader()`. Then you can get the rows of the CSV file as a sequence from the reader, e.g. with a `for` loop.

The tree cadastre data has already been saved in this repository. Alternatively you can also download it from the [open data portal of the city of Konstanz](https://offenedaten-konstanz.de/dataset/baumkataster-konstanz).

Let us try to read the tree cadastre from the CSV file.

In [ ]:
# open file
with open(file="data/KN_Baumkataster_2020.csv", mode="r") as csv_file:
    # create reader
    csv_reader = csv.reader(csv_file, delimiter=",")

    # iterate over rows in file
    for row in csv_reader:
        # print row
        print(row)

Another file format is JSON, which is a standardized and common data format to store and interchange data independent from any programming language. JSON data types are numbers, Unicode strings, boolean values, the `null` value (`None`), arrays (Python lists) and objects (Python dictionaries). The JSON data types and the JSON syntax are similar to Python. But there are subtle differences and we use the module `json` to read or write JSON data:

Let us first import the module.

In [ ]:
import json

We have prepared translations of the tree names in the tree cadastre. These translations were obtained from the [Wikispecies project](https://species.wikimedia.org/wiki/Main_Page) via the [Mediawiki API](https://www.mediawiki.org/wiki/API:Main_page). We will later learn how to use an [API](https://en.wikipedia.org/wiki/API) (Application Programming Interface) later.

Let us read the translations from the JSON file.

In [ ]:
# open file
with open(file="data/trees-wikispecies.json", mode="r") as json_file:
    # read json data
    json_data = json.load(json_file)

# print translations for one tree
json_data["Gleditsia triacanthos"]

### Pandas

Now you have seen how you can manually load the data from files. Maybe you have realized that this exercise is very time-consuming because many little steps must be implemented: How do I open the file? How do I iterate over the lines? How do I save it?

That is why developers have written external packages for data handling. For example, in these packages files can be loaded in one line of code. One of these packages is `pandas` and it supports all kinds of different data sources.

The package `pandas` helps you to arrange your data like tables. Through `pandas`, you can import, clean, transform, analyse and export data.

Let us first import the package or install it again if necessary.

In [ ]:
import pandas as pd

The primary component of `pandas` is its `DataFrame`. A `DateFrame` is organized like a table and has rows and columns. There are several ways to create a new `DataFrame`, but the easiest way to start is to take a dictionary and pass it into a new `DataFrame`.

When you want to load data from files, you can load it as `DataFrame` with `pandas` from a CSV or JSON files with the functions `pd.read_csv()` and `pd.read_json()`.

Let us load the tree cadastre with pandas.

In [ ]:
# read dataframe
trees = pd.read_csv("data/KN_Baumkataster_2020.csv")

# print dataframe
print(trees)

As your datasets become larger and non-trivial, you will not be able to print the entire data set anymore. `pandas` offers two methods `head()` and `tail()` to display either the first or the last rows of a `DataFrame`. By default, 10 rows are shown, but you can select any other number too.

Let us print the first and last row once.

In [ ]:
# print first row
print(trees.head(1))

In [ ]:
# print last row
print(trees.tail(1))

Instead of accessing the entire `DateFrame`, you can also restrict the selection to certain columns, rows and cells. Columns can be easily restricted with an index with a list of their names. Rows can either be selected by their name using the method `loc[]` or by their index using the method `iloc[]`. Of course you can also restrict both columns and rows at the same time.

Let us select different parts of our table.

In [ ]:
# select column of Name_dt
trees_selected = trees.loc[:, ["Name_dt"]]

# print selected dataframe
print(trees_selected)

In [ ]:
# select rows with tree baumId 39080
trees_selected = trees[trees["baumId"] == 1970]

# print selected dataframe
print(trees_selected)

When you want to select parts of a `DataFrame` which fulfill a certain condition, then you can also write a conditional statement instead of a name or index. You then write the statement again as a kind of index behind the `DataFrame`. Then you get all parts of the `DateFrame` for which the condition is correct. Of course you can also combine several conditions in round brackets `(` `)` with logical ands `&` and logical ors `|`.

Let us select trees according to their prices.

In [ ]:
# select rows with tree name Populus nigra 'Italica' and higher than 20 meters
trees_selected = trees[(trees["Name_lat"] == "Populus nigra 'Italica'") & (trees["hoeheM"] > 30)]

# print selected dataframe
print(trees_selected)

<div class="alert alert-block alert-info">
    <b>Exercise</b>: Inspect the dataframe. Select the trees with a height smaller than 20 meters and a trunk perimeter of more than 600 centimeters.
</div>

There are also several ways to change a `DataFrame` in `pandas`. A few ways are for example to append a `DataFrame` to another `DataFrame` with `append()`, find and delete duplicate rows with `drop_duplicates()`, sort rows with `sort_values()` and find rows with missing values and delete them with `dropna()`. Note that certain operations only pass a reference and others pass a new `DataFrame`. If you want to write to the exact same `DataFrame`, you can in most cases pass the argument `inplace = True` to the function.

Let us clean up the data a little bit by translating the German column names, dropping the columns not used later on and using the column "OBJECTID" as row index

In [ ]:
# define columns rename
columns_rename = {"hoeheM": "height (m)", "kronendurchmesserM": "treetop diameter (m)", "stammumfangCM": "trunk perimeter (cm)"}

# define columns drop
columns_drop = ["baumId", "baumNr", "baumart", "AGOL_Name"]

# rename columns in dataframe
trees.rename(columns=columns_rename, inplace=True)

# drop columns in dataframe
trees.drop(columns=columns_drop, inplace=True)

# set index in dataframe
trees.set_index("OBJECTID", inplace=True)

# print head of dataframe
trees.head()

## 2.3 Data Analysis

On the basis of the completed data handling, the next step in our data pipeline is to start with the data analysis. Descriptive statistics help us in the first place to better understand the data, e.g. means, quantiles, deviations, counts etc. These are implemented in `pandas`. To model the data and recognize mechanisms we move on to inferential statistics, e.g. correlations, regressions etc. For these statistics we will introduce `sklearn`. But let us take a look at all packages one after the other.

### Pandas

Within a `DateFrame` of `pandas`, you can use the method `describe()` to view the basic statistical characteristics of each feature. These characteristics can also be calculated individually using e.g. the methods `max()`, `min()` and `mean()`. Or additionally calculate the methods `sum()` and `corr()`, which should explain themselves by their names.

Let us have a look on the basic statistical characteristics of our features.

In [ ]:
# print characteristics
print(trees.describe())

In [ ]:
# define columns numeric
columns_numeric = ["trunk perimeter (cm)", "height (m)", "treetop diameter (m)"]

# print correlations
print(trees.loc[:, columns_numeric].corr())

In [ ]:
# count tree names and show the N most frequent tree names
trees_top = trees["Name_lat"].value_counts().head(20).to_frame()

# print top trees
print(trees_top)

## Sklearn

A new package, we have not seen yet, is `sklearn` or `scikit-learn`. `sklearn` offers different machine learning methods like linear models, support vector machines, tree-based methods, nearest neighbors, neural networks, clustering, matrix decomposition... Each of these methods can be conveniently prepared, performed and evaluated. To demonstrate how easy an analysis in `sklearn` is, we will run a linear regression. First, we will demonstrate it on our trivial data set, but then you will do it by yourself on your data set.

Let us first import from the package its linear model or install it again if necessary.

In [ ]:
from sklearn import linear_model

The first step is to prepare the underlying data as `numpy` array, where the dependent variable is usually stored in `Y`, and the independent variables are stored in `X`. The next step is to create a linear regression model using `sklearn`. This model can be computed by passing the dependent and independent variables with the method `fit()`. To get some results of the model, you can retrieve the intercept from `intercept_`, the coefficients from `coef_`, and all kinds of errors.

Let us run our first linear regression.

In [ ]:
# keep trees without nans
trees_notna = trees.dropna(subset=["height (m)", "trunk perimeter (cm)"])

# prepare dependent variable
Y = trees_notna["height (m)"].to_numpy()

# prepare independent variable
X = trees_notna["trunk perimeter (cm)"].to_numpy().reshape(-1, 1)

# create linear regression model
regression = linear_model.LinearRegression()

# compute linear regression model
regression.fit(X, Y)

# print intercept
print("Intercept is", regression.intercept_)

# print coefficients
print("Coefficients are", regression.coef_)

## 2.3 Data Visualization

### Matplotlib

The package `matplotlib` is de facto the visualization tool in Python. It allows us to create all possible kinds of visualizations and get further insights into our data. You can basically use `matplotlib` out of `pandas`, but we will use it as a standalone package for the sake of clarity.

A figure in `matplotlib` consists in principle of the following components:

| Component | Description |
| -------- | ------- |
| Figure | canvas which contains one or more axes |
| Axes | plot with one axis per dimension |
| Axis | number line like object |

Let us first make the necessary imports and choose the [matplotlib's style "ggplot"](https://matplotlib.org/stable/gallery/style_sheets/ggplot.html).

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("ggplot")

For our first plot, we create a first trivial scatter plot of the 3 metric values. For that, we call on the dataframe `trees` the method `plot()`, which creates the actual plot. With the method `show()` we invoke all previously created plots.

Let us create our first plot.

In [ ]:
# create plot
trees.plot(kind="scatter", x="trunk perimeter (cm)", y="height (m)", s="treetop diameter (m)")

# show plot
plt.show()

For our next plot, we will take into account the tree types. We will focus on the top-20 most frequent names only and plot the metrics per tree on a 4x5 matrix and add some color to the plots.

Let us create our next plot.

In [ ]:
# create plot as grid of subplots
fig, axes = plt.subplots(nrows=5, ncols=4, sharex=True, sharey=True, squeeze=False, figsize=[20,25])

# iterate over top-20 most frequent names and subplots
n = 0
for tree in trees_top.index.to_list():
    trees[trees["Name_lat"]==tree].plot(
        kind="scatter",
        ax=axes[int(n/4),n%4],
        title=tree,
        x="trunk perimeter (cm)",
        y="height (m)",
        s="treetop diameter (m)", # show by point size
        c="treetop diameter (m)", # also indicated by color
        colormap="Spectral",
        norm=matplotlib.colors.LogNorm(vmin=1, vmax=25),
        colorbar=None)
    n += 1

# show plot
plt.show()

### Folium

The package `folium` allows us to plot geographic data, as the ones from the trees in the tree cadastre of the city Konstanz. In this dataset, every tree has a longitude and latitude. The locations of the trees can be visualized in an interactive map using `folium`.

Let us import the necessary modules.

In [ ]:
import folium
import branca.colormap as cm

To create an interactive map, we create a map `folium.Map`. In this map, each tree will have a marker where the color of the marker depends on the height and the radius of the marker on the diameter.

Let us create this interactive map.

In [ ]:
# create map
map = folium.Map(location=[47.66336, 9.17598], tiles="Stamen Terrain", zoom_start=16)

# create colormap
colormap = cm.LinearColormap(colors=["lightgreen", "darkgreen"], vmin=1, vmax=40).to_step(n=12)

# define helper function for color
def color_height(height):
    if 1.0 <= height <= 40.0:
        return colormap(height)
    else:
        return "darkblue"

# define helper function to add trees to map
def map_tree(row):
    marker = folium.CircleMarker(
        location=(row["Y"], row["X"]),
        tooltip=folium.Tooltip(row["Name_lat"]),
        radius=row["treetop diameter (m)"]/4,
        fill=True,
        color=color_height(row["height (m)"]),
    )
    marker.add_to(map)

# add trees to map
trees.apply(map_tree, axis=1)

# modify map by color height
map.add_child(colormap, name="height (m)")

# show map
map